In [1]:
import numpy as np
import pickle
import pandas as pd
from sklearn import linear_model
from sklearn.metrics import roc_auc_score, classification_report, roc_curve
import warnings
warnings.filterwarnings('ignore')

In [2]:
input_path = 'E:/CS_Master_Degree_UIUC/CS598_DeepLearning_for_Health_Data/Project/paper290/MIMIC_Processed/'
data_path = 'E:/CS_Master_Degree_UIUC/CS598_DeepLearning_for_Health_Data/Project/paper290/MIMIC data/'
output_path = 'E:/CS_Master_Degree_UIUC/CS598_DeepLearning_for_Health_Data/Project/paper290/Output/'

In [3]:
# The vocabulary sizes were all updated per preprocessed results
vocabsize_icd = 942
vocabsize_fullicd = 4894 # full version codes 
vocabsize_meds = 3202
vocabsize_labs = 284 #all lab is 681
vocabsize = vocabsize_icd+vocabsize_meds+vocabsize_labs # 4428

input_seqs_icd = np.array(pickle.load(open(input_path +'MIMICIIIPROCESSED.3digitICD9.seqs', 'rb'))) #short version diag icd
input_seqs_meds = np.array(pickle.load(open(input_path +'MIMICIIIPROCESSED.meds.seqs', 'rb')))
input_seqs_labs = np.array(pickle.load(open(input_path +'MIMICIIIPROCESSED.abnlabs.seqs', 'rb')))
input_seqs_fullicd = np.array(pickle.load(open(input_path +'MIMICIIIPROCESSED.seqs', 'rb'))) #full version diag icd
labels = np.array(pickle.load(open(input_path +'MIMICIIIPROCESSED.morts', 'rb')))

In [4]:
# fout = open("logistic_regression_interpretations.txt", 'w')

def combine_encounter(seqs, vocab):
    ret_vector = np.zeros([len(seqs), vocab])
    for i, enc in enumerate(seqs):
#         print(i)
        for code in enc:
            ret_vector[i, code] = 1
    return ret_vector.sum(axis = 0) #modified to sum up all codes across visits for each patient

In [5]:
# for modeling using only diagnoses icd9 feature
input_icds = np.array([combine_encounter(input_seqs_icd[i], vocabsize_icd) for i in range(0, len(input_seqs_icd))])

# for modeling using only med feature
input_meds = np.array([combine_encounter(input_seqs_meds[i], vocabsize_meds) for i in range(0, len(input_seqs_meds))])

# for modeling using only abnormal lab feature
input_labs = np.array([combine_encounter(input_seqs_labs[i], vocabsize_labs) for i in range(0, len(input_seqs_labs))])

# for modeling with concatenated features
input_seqs = np.array([np.concatenate((combine_encounter(input_seqs_icd[i], vocabsize_icd), 
                                       combine_encounter(input_seqs_meds[i], vocabsize_meds), 
                                       combine_encounter(input_seqs_labs[i], vocabsize_labs)), axis=0) for i in range(0, len(input_seqs_icd))])


In [6]:
trainratio = 0.7
validratio = 0.1
testratio = 0.2

trainlindex = int(len(input_seqs_icd)*trainratio)
validlindex = int(len(input_seqs_icd)*(trainratio + validratio))

### Modeling with only diagnoses ICD codes

In [7]:
best_aucrocs = []
for run in range(10):
    print('Run', run)

    perm = np.random.permutation(input_icds.shape[0])
    rinput_seqs = input_icds[perm]
    rlabels = labels[perm]

    train_input_seqs = rinput_seqs[:trainlindex]
    train_labels = rlabels[:trainlindex]

    valid_input_seqs = rinput_seqs[trainlindex:validlindex]
    valid_labels = rlabels[trainlindex:validlindex]

    test_input_seqs = rinput_seqs[validlindex:]
    test_labels = rlabels[validlindex:]

    model = linear_model.LogisticRegression(solver='liblinear')

    model.fit(train_input_seqs, train_labels)

    vpredict_probabilities = np.array([a[1] for a in model.predict_proba(valid_input_seqs)])
    print("Validation AUC_ROC: ", roc_auc_score(valid_labels, vpredict_probabilities))

    predict_probabilities = np.array([a[1] for a in model.predict_proba(test_input_seqs)])
    print("Test AUC_ROC: ", roc_auc_score(test_labels, predict_probabilities))

    best_aucrocs.append(roc_auc_score(test_labels, predict_probabilities))

print("Average AUCROC:", np.mean(best_aucrocs), "+/-", np.std(best_aucrocs))

Run 0
Validation AUC_ROC:  0.7855861889347769
Test AUC_ROC:  0.7482075660751264
Run 1
Validation AUC_ROC:  0.7745656170313705
Test AUC_ROC:  0.77542061089181
Run 2
Validation AUC_ROC:  0.7653950312021663
Test AUC_ROC:  0.773814934784735
Run 3
Validation AUC_ROC:  0.7963422099785737
Test AUC_ROC:  0.7651248715170709
Run 4
Validation AUC_ROC:  0.7749523061384805
Test AUC_ROC:  0.7837432148000444
Run 5
Validation AUC_ROC:  0.7870364082248632
Test AUC_ROC:  0.7714525599140984
Run 6
Validation AUC_ROC:  0.7742233136138705
Test AUC_ROC:  0.7876078530647627
Run 7
Validation AUC_ROC:  0.7524586917779588
Test AUC_ROC:  0.7872235973907744
Run 8
Validation AUC_ROC:  0.7729547497752471
Test AUC_ROC:  0.7707651429122784
Run 9
Validation AUC_ROC:  0.7718317298622901
Test AUC_ROC:  0.7698134860483676
Average AUCROC: 0.7733173837399069 +/- 0.011094405645520578


### Modeling with only medication feature

In [8]:
best_aucrocs = []
for run in range(10):
    print('Run', run)

    perm = np.random.permutation(input_meds.shape[0])
    rinput_seqs = input_meds[perm]
    rlabels = labels[perm]

    train_input_seqs = rinput_seqs[:trainlindex]
    train_labels = rlabels[:trainlindex]

    valid_input_seqs = rinput_seqs[trainlindex:validlindex]
    valid_labels = rlabels[trainlindex:validlindex]

    test_input_seqs = rinput_seqs[validlindex:]
    test_labels = rlabels[validlindex:]

    model = linear_model.LogisticRegression(solver='liblinear')

    model.fit(train_input_seqs, train_labels)

    vpredict_probabilities = np.array([a[1] for a in model.predict_proba(valid_input_seqs)])
    print("Validation AUC_ROC: ", roc_auc_score(valid_labels, vpredict_probabilities))

    predict_probabilities = np.array([a[1] for a in model.predict_proba(test_input_seqs)])
    print("Test AUC_ROC: ", roc_auc_score(test_labels, predict_probabilities))

    best_aucrocs.append(roc_auc_score(test_labels, predict_probabilities))

print("Average AUCROC:", np.mean(best_aucrocs), "+/-", np.std(best_aucrocs))

Run 0
Validation AUC_ROC:  0.7188406986547689
Test AUC_ROC:  0.7431656001717029
Run 1
Validation AUC_ROC:  0.7748584328129783
Test AUC_ROC:  0.7516159438549337
Run 2
Validation AUC_ROC:  0.7594228979689146
Test AUC_ROC:  0.7653681586486586
Run 3
Validation AUC_ROC:  0.7727487312640152
Test AUC_ROC:  0.7510371954537212
Run 4
Validation AUC_ROC:  0.7424789410348976
Test AUC_ROC:  0.7362203615218176
Run 5
Validation AUC_ROC:  0.7531959360507688
Test AUC_ROC:  0.7581038220842051
Run 6
Validation AUC_ROC:  0.7343302932761088
Test AUC_ROC:  0.7691278999879794
Run 7
Validation AUC_ROC:  0.759790899281914
Test AUC_ROC:  0.7526459854014598
Run 8
Validation AUC_ROC:  0.7474435376731828
Test AUC_ROC:  0.7423462415007984
Run 9
Validation AUC_ROC:  0.7652745729668806
Test AUC_ROC:  0.7688263505900493
Average AUCROC: 0.7538457559215326 +/- 0.01086332198498757


### Modeling with only lab feature

In [9]:
best_aucrocs = []
for run in range(10):
    print('Run', run)

    perm = np.random.permutation(input_labs.shape[0])
    rinput_seqs = input_labs[perm]
    rlabels = labels[perm]

    train_input_seqs = rinput_seqs[:trainlindex]
    train_labels = rlabels[:trainlindex]

    valid_input_seqs = rinput_seqs[trainlindex:validlindex]
    valid_labels = rlabels[trainlindex:validlindex]

    test_input_seqs = rinput_seqs[validlindex:]
    test_labels = rlabels[validlindex:]

    model = linear_model.LogisticRegression(solver='liblinear')

    model.fit(train_input_seqs, train_labels)

    vpredict_probabilities = np.array([a[1] for a in model.predict_proba(valid_input_seqs)])
    print("Validation AUC_ROC: ", roc_auc_score(valid_labels, vpredict_probabilities))

    predict_probabilities = np.array([a[1] for a in model.predict_proba(test_input_seqs)])
    print("Test AUC_ROC: ", roc_auc_score(test_labels, predict_probabilities))

    best_aucrocs.append(roc_auc_score(test_labels, predict_probabilities))

print("Average AUCROC:", np.mean(best_aucrocs), "+/-", np.std(best_aucrocs))

Run 0
Validation AUC_ROC:  0.7383333333333333
Test AUC_ROC:  0.7177836805070135
Run 1
Validation AUC_ROC:  0.7145222536206287
Test AUC_ROC:  0.7079871432818075
Run 2
Validation AUC_ROC:  0.7192052576667961
Test AUC_ROC:  0.7276732650317557
Run 3
Validation AUC_ROC:  0.7338188004088667
Test AUC_ROC:  0.7190369358275841
Run 4
Validation AUC_ROC:  0.6826060031784166
Test AUC_ROC:  0.7167023177857414
Run 5
Validation AUC_ROC:  0.7277167899365209
Test AUC_ROC:  0.7101505563044024
Run 6
Validation AUC_ROC:  0.7393374463519312
Test AUC_ROC:  0.7269089771101573
Run 7
Validation AUC_ROC:  0.7230281845666462
Test AUC_ROC:  0.7302502446525933
Run 8
Validation AUC_ROC:  0.6869915567607543
Test AUC_ROC:  0.7351961950059452
Run 9
Validation AUC_ROC:  0.736097804008738
Test AUC_ROC:  0.7108049895115373
Average AUCROC: 0.7202494305018537 +/- 0.008857569765513313


### Modeling with concatenated features (diagnoses icds +  medications + abnormal lab components)

In [17]:
best_aucrocs = []
for run in range(10):
    print('Run', run)

    perm = np.random.permutation(input_seqs.shape[0])
    rinput_seqs = input_seqs[perm]
    rinput_seqs_fullicd = input_seqs_fullicd[perm]
    rlabels = labels[perm]
    r_input_icd = input_seqs_icd[perm]

    train_input_seqs = rinput_seqs[:trainlindex]
    train_input_seqs_fullicd = rinput_seqs_fullicd[:trainlindex]
    train_labels = rlabels[:trainlindex]

    valid_input_seqs = rinput_seqs[trainlindex:validlindex]
    valid_input_seqs_fullicd = rinput_seqs_fullicd[trainlindex: validlindex]
    valid_labels = rlabels[trainlindex:validlindex]

    test_input_seqs = rinput_seqs[validlindex:]
    test_input_seqs_fullicd = rinput_seqs_fullicd[validlindex:]
    test_labels = rlabels[validlindex:]
    test_input_seqs_interpretations = r_input_icd[validlindex:]

    model = linear_model.LogisticRegression(solver='liblinear')
    
    model.fit(train_input_seqs, train_labels)

    vpredict_probabilities = np.array([a[1] for a in model.predict_proba(valid_input_seqs)])

    print("Validation AUC_ROC: ", roc_auc_score(valid_labels, vpredict_probabilities))

    predict_probabilities = np.array([a[1] for a in model.predict_proba(test_input_seqs)])
    
    print("Test AUC_ROC: ", roc_auc_score(test_labels, predict_probabilities))

    best_aucrocs.append(roc_auc_score(test_labels, predict_probabilities))

print("Average AUCROC:", np.mean(best_aucrocs), "+/-", np.std(best_aucrocs))

Run 0
Validation AUC_ROC:  0.780855571583123
Test AUC_ROC:  0.7876151902497028
Run 1
Validation AUC_ROC:  0.7489889163369259
Test AUC_ROC:  0.7913267979460212
Run 2
Validation AUC_ROC:  0.7724186566436347
Test AUC_ROC:  0.772575748164294
Run 3
Validation AUC_ROC:  0.7655968265416515
Test AUC_ROC:  0.7920272741406499
Run 4
Validation AUC_ROC:  0.7987079756795421
Test AUC_ROC:  0.7831191487107296
Run 5
Validation AUC_ROC:  0.7703435804701628
Test AUC_ROC:  0.7857223159078133
Run 6
Validation AUC_ROC:  0.7702848519147332
Test AUC_ROC:  0.8023892866862491
Run 7
Validation AUC_ROC:  0.7827229827229827
Test AUC_ROC:  0.7853846628301001
Run 8
Validation AUC_ROC:  0.8034595498783454
Test AUC_ROC:  0.7847877758913412
Run 9
Validation AUC_ROC:  0.799153431225652
Test AUC_ROC:  0.7894439837847798
Average AUCROC: 0.787439218431168 +/- 0.007193767941846893


### Interpretation of the modeling results

1. Map integer ID of lab, meds and icd codes to the name of the code

In [20]:
def get_ICD(icd):
    '''
    Given icd integer index, return the string name of that icd code:
    e.g. get_ICD(1) returns "Hypertension NOS"
    '''
    ret_str = ""
    icd_key_lst = list(icditems.keys())
    icd_val_ind = list(icditems.values())[icd]
    icd_str = icd_key_lst[icd_val_ind]
    actual_key = icd_str.replace(".", "")[2:]
    if actual_key in icdnames:
        ret_str = icdnames[actual_key]
    else:
        ret_str = icd_str
    return ret_str

def get_med(med):
    '''
    Given icd integer index, return the string name of that med code:
    e.g. get_med(1) returns "Phenylephrine HCl"
    '''
    med_key_lst = list(meditems.keys())
    med_val_ind = list(meditems.values())[med]
    ret_str = med_key_lst[med_val_ind]
    return ret_str

def get_lab(lab):
    '''
    Given lab integer index, return the string name of that lab code:
    e.g. get_lab(1) returns "Hemoglobin"
    '''
    lab_key_lst = list(labitems.keys())
    lab_val_ind = list(labitems.values())[lab]
    ret_str = labnames[lab_key_lst[lab_val_ind]]
    return ret_str

def get_factor(i, patid):
    if i<vocabsize_icd:
        return get_ICD(pattofullicd_dict[patid][i]), 0
    elif i<vocabsize_icd+vocabsize_meds:
        return get_med(i-vocabsize_icd), 1
    else:
        return get_lab(i-vocabsize_icd-vocabsize_meds), 2

In [21]:
labnames = {}
lab_dict_file = open(data_path + 'D_LABITEMS.csv', 'r')

lab_dict_file.readline()
for line in lab_dict_file:
    tokens = line.strip().split(',')
    labnames[tokens[1].replace('"','')] = tokens[2]
lab_dict_file.close()

In [22]:
icdnames = {}
icd_dict_file = open(data_path + 'D_ICD_DIAGNOSES.csv', 'r')
icd_dict_file.readline()
for line in icd_dict_file:
    tokens = line.strip().split(',')
    icdnames[tokens[1].replace('"','')] = tokens[2]
icd_dict_file.close()

2. Find the top 10 risk ICD factors for patient having predicted death

In [23]:
icditems = pickle.load(open(input_path + 'MIMICIIIPROCESSED.types', 'rb'))
meditems = pickle.load(open(input_path + 'MIMICIIIPROCESSED.meds.types', 'rb'))
labitems = pickle.load(open(input_path + 'MIMICIIIPROCESSED.abnlabs.types', 'rb'))

In [25]:
interpretation_file = open(output_path + "Log_Reg_Interpretations.txt", 'w')

pattofullicd_dict = {}
for i in range(len(test_input_seqs_interpretations)):
    icdtofullicd_dict = {}
    for j in range(len(test_input_seqs_interpretations[i])):
        for k in range(len(test_input_seqs_interpretations[i][j])):
            icdtofullicd_dict[test_input_seqs_interpretations[i][j][k]] = test_input_seqs_fullicd[i][j][k]
    pattofullicd_dict[i] = icdtofullicd_dict

coeffs = np.array(model.coef_[0])
for patid in range(len(test_input_seqs)):
    test_input = test_input_seqs[patid]

    scores = (test_input*coeffs)
    # scores = coeffs
    risk_scores = []
    for i in range(len(scores)):
        if test_input[i]>0:
            factors = get_factor(i, patid)
            risk_scores.append((factors[0], scores[i]))
    risk_scores.sort(key=lambda tup: tup[1], reverse=True)

    top_risk_factors = risk_scores[:10]

    if (predict_probabilities[patid] > 0.5):
        interpretation_file.write("ID: " + str(patid) + " True label: "+str(test_labels[patid])+"\n")
        for rf in top_risk_factors:
            interpretation_file.write(str(rf)+"\n")
        interpretation_file.write("\n")

interpretation_file.close()

In [35]:
f = pd.read_fwf(output_path + "Log_Reg_Interpretations.txt", header = None)
f.head(30)

,0
0,ID: 1 True label: 1
1,"('""Oth sequela', 2.2793366625328755)"
2,"('""Scopolamine Patch""', 1.6849271049755468)"
3,"('""Ascites NEC""', 0.8586911454155374)"
4,"('""Intracerebral hemorrhage""', 0.7548940425002..."
5,"('""Lymphocytes""', 0.6439327688056661)"
6,"('""Polys""', 0.6041732650514918)"
7,"('""INR(PT)""', 0.5344781598355357)"
8,"('""Red Blood Cells""', 0.47128239198661176)"
9,"('""Venous insufficiency NOS""', 0.4163467988504..."
